<a href="https://colab.research.google.com/github/gabrielfernandorey/ITBA-NLP/blob/main/ITBA_nlp01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Practico NLP - Detección de Tópicos y clasificación
- ITBA 2024
- Alumno: Gabriel Rey
---

### Resumen del problema

- Calcular los tópicos de portales de noticias que se reciben 
- Frecuencia del cálculo de tópicos: diaria
- Colección de noticias: diariamente, en lotes o de a un texto.
- Identificar tópicos, entidades, keywords y análisis de sentimiento.

### Datos
- Se reciben las noticias con formato: Titulo, Texto, Fecha, Entidades, Keywords

### Tareas
- Modelo de detección de tópicos diario utilizando embeddings
- Definir un criterio de agrupación de tópicos aplicado al mismo día y entre distintos días (merging)
- Almacenar los embeddings de tópicos en una base de datos vectorial
- Modelo de datos dado: 
    - Id del tópico
    - Nombre del tópico
    - Keywords
    - Embbeding
    - Fecha de creación
    - Fecha de entrenamiento inicial
    - Fecha de entrenamiento actualizada
    - Umbral de detección
    - Documento mas cercano
---
Tareas en esta notebook:
- Inicializar la base de datos vectorial
- Ingestar data
- NER: Encontrar las entidades de cada documento
- Limpiar data
- Modelo: Armado del modelo BERTopic
- Entrenamiento
- Almacenamiento en base de datos vectorial


In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import re
import json
from datetime import datetime
from dotenv import load_dotenv
from tqdm import tqdm
from collections import Counter

import spacy

from NLP_tools import clean_all
from core.functions import *

# -->> levantar la base antes de ejecutar
from opensearch_data_model import os_client
from opensearch_io import init_opensearch

### Inicializamos la base vectorial
Se modifica la indice de la base "Topic" agregando referencias del documento mas cercano y un campo para los 100 documentos mas cercanos al tópico.

In [2]:
# Inicialización de indices
init_opensearch()

2024-07-21 19:57:37.739 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`


El índice Topic ya existe. Saltando inicialización de base de datos.
El índice News ya existe. Saltando inicialización de base de datos.


### Path

In [3]:
load_dotenv()
PATH_REMOTO='/content/ITBA-NLP/data/'
PATH=os.environ.get('PATH_LOCAL', PATH_REMOTO)
PATH

'C:/Users/gabri/OneDrive/Machine Learning/Github/ITBA-NLP/data/'

### Data

In [4]:
# Read the parquet file | ( lotes de prueba )

df_params = {'0_1000':'0_1000_data.parquet',
             '1000_2000':'1000_2000_data.parquet',
             '2000_3000':'2000_3000_data.parquet',
             'df_joined':'df_joined_2024-04-01 00_00_00.parquet'
            }

#chunk = os.environ.get('CHUNK')
chunk = '0_1000'
print(chunk)

df_parquet = pd.read_parquet(PATH+df_params[chunk])

data = list(df_parquet['in__text'])

df_parquet.head(1)

0_1000


,Asset Name,Author Id,Author Name,Keyword Id,Keyword Name,Entity Id,Entity Name,Media Group Id,Media Group Name,Impact,...,in__text,out__entities,out__potential_entities,predicted_at_entities,out__keywords_sorted,predicted_at_keywords,start_time_utc,start_time_local,truncated_text,title_and_text
Asset Id,,,,,,,,,,,,,,,,,,,,,
105628101,Elecciones en Venezuela: María Corina Machado ...,36192,Infobae,1932002 | 417739 | 1687638 | 36187 | 7476 | 50...,"[falsas conspiraciones armadas, sustituta, det...",219925 | 210613 | 219770 | 36424 | 1129437,"[Nicolás Maduro, Jorge Rodríguez, Marcelo Ebra...",0,,7406333,...,Fotografía de archivo de la líder antichavista...,"[Nicolás Maduro, Marcelo Ebrard, Jorge Rodrígu...","[Jorge Rodríguez, Nicolás Maduro, Rayner Peña ...",2024-04-02 08:11:57.825777,"[elecciones presidenciales, candidatura presid...",2024-04-02 08:17:44.372891+00:00,2024-04-02,2024-04-01 21:00:00,Fotografía de archivo de la líder antichavista...,Elecciones en Venezuela: María Corina Machado ...


In [5]:
# Cantidad total de documentos
len(data)

1000

### StopWords

In [5]:
# Stopwords
SPANISH_STOPWORDS = list(pd.read_csv(PATH+'spanish_stop_words.csv' )['stopwords'].values)
SPANISH_STOPWORDS_SPECIAL = list(pd.read_csv(PATH+'spanish_stop_words_spec.csv' )['stopwords'].values)

In [ ]:
""" import csv
# Guardar la lista de stopwords especial en un archivo CSV
with open(PATH+"spanish_stop_words_spec.csv", mode='w', newline='', encoding='utf-8') as archivo:
    escritor = csv.writer(archivo)
    escritor.writerow(['stopwords'])
    for stopword in SPANISH_STOPWORDS_SPECIAL:
        escritor.writerow([stopword]) """

### NER - Named Entity Recognition
Obtener entidades de las noticias 

In [6]:
# Cargar el modelo de spaCy para español
spa = spacy.load("es_core_news_lg")

In [24]:
 
# Cargar o saltar carga y procesar celda inferior
with open(PATH+f'modelos/entities_{chunk}.json', 'r') as json_file:
    entities = json.load(json_file)

with open(PATH+f'modelos/keywords_{chunk}.json', 'r') as json_file:
    keywords = json.load(json_file)

with open(PATH+f'modelos/vocabulary_{chunk}.json', 'r') as json_file:
    vocab = json.load(json_file) 

In [12]:
# Detectar entidades para todos los documentos usando spaCy

entities = []
for data_in in tqdm(data):

    # Contabilizar palabras en doc original
    normalized_text = re.sub(r'\W+', ' ', data_in.lower())
    words_txt_without_stopwords = [word for word in normalized_text.split() if word not in SPANISH_STOPWORDS+SPANISH_STOPWORDS_SPECIAL]
    words_txt_counter = Counter(words_txt_without_stopwords)
    words_counter = {elemento: cuenta for elemento, cuenta in sorted(words_txt_counter.items(), key=lambda item:item[1], reverse=True) if cuenta > 1}

    # Extraer entidades del doc segun atributos
    extract = spa(data_in)
    entidades_spacy = [(ent.text, ent.label_) for ent in extract.ents]
    ent_select = [ent for ent in entidades_spacy if ent[1] == 'PER' or ent[1] == 'ORG' or ent[1] == 'LOC' ]

    # Extraer entidades de "maximo 3 palabras"
    ent_max_3 = [ent[0] for ent in ent_select if len(ent[0].split()) <= 3]
    ent_clean = clean_all(ent_max_3, accents=False)
    ent_unique = list(set([ word for word in ent_clean if word not in SPANISH_STOPWORDS+SPANISH_STOPWORDS_SPECIAL] ))

    ents_proc = {}
    for ent in ent_unique:
        
        # Criterio de selección 
        weight = 0
        for word in ent.split():
            if word in words_counter:
                weight += 1 /len(ent.split()) * words_counter[word]
        
        ents_proc[ent] = round(weight,4)

    ents_proc_sorted = {k: v for k, v in sorted(ents_proc.items(), key=lambda item: item[1], reverse=True) if v > 0}

    # Crear la lista preliminar de entidades procesadas por noticia 
    pre_entities = [key for key, _ in ents_proc_sorted.items()] 

    # Obtener las últimas palabras de cada entidad que tenga mas de una palabra por entidad
    last_words = list(set([ent.split()[-1] for ent in pre_entities if len(ent.split()) > 1 ]))

    # Eliminar palabra única si la encuentra al final de una compuesta
    pre_entities_without_last_word_equal = []
    for idx, ent in enumerate(pre_entities):
        if not (len(ent.split()) == 1 and ent in last_words):
            pre_entities_without_last_word_equal.append(ent)

    # Obtener las palabras únicas
    unique_words = [ ent.split()[0] for ent in pre_entities_without_last_word_equal if len(ent.split()) > 1 ]

    # Eliminar palabra única si la encuentra al comienzo de una compuesta
    pre_entities_without_first_word_equal = []
    for idx, ent in enumerate(pre_entities_without_last_word_equal):
        if not (len(ent.split()) == 1 and ent in unique_words):
            pre_entities_without_first_word_equal.append(ent)

    # obtener entidades filtradas
    if len(pre_entities_without_first_word_equal) > 10:
        umbral = 10 + (len(pre_entities_without_first_word_equal)-10) // 2
        filter_entities = pre_entities_without_first_word_equal[:umbral] 
    else:
        filter_entities = pre_entities_without_first_word_equal[:10]

    pre_original_entities = []
    # capturar las entidades en formato original
    for ent in filter_entities:
        pre_original_entities.append([elemento for elemento in ent_max_3 if elemento.lower() == ent.lower()])

    sort_original_entities = sorted(pre_original_entities, key=len, reverse=True)
    
    try:
        entities.append( [ent[0] for ent in sort_original_entities if ent] ) 
    except Exception as e:
        entities.append([])


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [01:54<00:00,  8.75it/s]


In [13]:
len(entities)

1000

In [14]:
# Grabar
with open(PATH+f'modelos/entities_{chunk}.json', 'w') as file:
    json.dump(entities, file)

## Keywords
Obtener palabras clave de las noticias

In [15]:
# Detectar keywords para todos los documentos usando spaCy

keywords_spa = []
for doc in tqdm(data):
    extract = spa(doc)
    keywords_spa.append([(ext.text, ext.pos_) for ext in extract])  

100%|██████████| 1000/1000 [01:44<00:00,  9.53it/s]


### Keyboards with neighboards

In [16]:
# Funcion para obtener keywords con combinaciones de bigramas
def get_bigrams(word_list, number_consecutive_words=2):
    
    ngrams = []
    adj_length_of_word_list = len(word_list) - (number_consecutive_words - 1)
    
    for word_index in range(adj_length_of_word_list):
        
        # Indexar la lista 
        ngram = word_list[word_index : word_index + number_consecutive_words]
        
        # Agregar a la lista de "ngrams"
        ngrams.append(ngram)
        
    return ngrams

In [17]:
# devolver las palabras más frecuentes que aparecen junto a una palabra clave en particular
def get_neighbor_words(keyword, bigrams, pos_label = None):
    
    neighbor_words = []
    keyword = keyword.lower()
    
    for bigram in bigrams:
        
        # Extrae solo las palabras en minúsculas (no las etiquetas) para cada bigrama
        words = [word.lower() for word, label in bigram]        
        
        # Comprueba si la palabra clave está en el bigram
        if keyword in words:
            idx = words.index(keyword)
            for word, label in bigram:
                
                #Ahora nos centramos en la palabra vecina, no en la palabra clave
                if word.lower() != keyword:
                    #Si la palabra vecina coincide con la pos_label correcta, agregarla a la lista maestra
                    if label == pos_label or pos_label == None:
                        if idx == 0:
                            neighbor_words.append(" ".join([keyword, word.lower()]))
                        else:
                            neighbor_words.append(" ".join([word.lower(), keyword]))
                    
    return Counter(neighbor_words).most_common()

In [18]:
def keywords_with_neighboards(keywords_spa, POS_1='NOUN', POS_2='ADJ'):
    """
    Funcion que devuelve dos listas:
    - lista de keywords with neighboards (segun argumentos POS_1 y POS_2)
    - lista de keywords mas frecuentes (segun argumentos POS_1 y POS_2)
    """

    doc_kwn = []
    commons = []
    for keywords in keywords_spa:
    
        # Obtenemos las keywords del tipo (Universal Dependences) mas frecuentes de cada doc (spaCy format)
        words = []
        for k_spa in keywords:
            if k_spa[1] == POS_1:
                words.append(k_spa[0])

        cont_words = Counter(words)

        common = cont_words.most_common()
        commons.append( [com for com in common if com[1] > 1] )

        # Calcular un umbral de corte (en repeticiones) para los keywords obtenidos
            ## suma de todos los valores
        valores = [valor for _, valor in common]

            ## Calcular los pesos como proporcionales a los valores mismos
        pesos = np.array(valores) / np.sum(valores)

            ## Calcular el umbral ponderado, valor 2 o superior ( debe repetirse la keyword al menos una vez )
        threshold = max(2, round(np.sum(np.array(valores) * pesos),4))


        # Obtenemos los bigramas del doc        
        tokens_and_labels = [(token[0], token[1]) for token in keywords if token[0].isalpha()]

        bigrams = get_bigrams(tokens_and_labels)

        keywords_neighbor = []
        for item_common in common:
            if item_common[1] >= threshold or len(keywords_neighbor) < 6: # corte por umbral o menor a 6
                
                kwn = get_neighbor_words(item_common[0], bigrams, pos_label=POS_2)
                if kwn != []:
                    keywords_neighbor.append( kwn )

        sorted_keywords_neighbor = sorted([item for sublist in keywords_neighbor for item in sublist ], key=lambda x: x[1], reverse=True)
        
        doc_kwn.append(sorted_keywords_neighbor)

    return doc_kwn, commons

In [19]:
# obtenemos keywords with neighboards y keywords mas frecuentes
k_w_n, keyword_single = keywords_with_neighboards(keywords_spa)

In [ ]:
k_w_n[4]

In [20]:
# filtramos las que al menos se repiten una vez
filtered_k_w_n = [ [tupla[0] for tupla in sublista if tupla[1] > 1] for sublista in k_w_n ]
filtered_k_w_n[1]


['ahorro interno',
 'sistema bancario',
 'descomunal tasa',
 'impuestos confiscatorios']

In [ ]:
# Analizamos los keywords unigrama
keyword_single[1]

In [21]:
# Si un keyword unigrama coincide en los bigramas elegidos se descarta
# la cantidad de keywords se obtiene utilizando la media como umbral de corte

# Umbral
values = [value for sublist in keyword_single for _, value in sublist]
threshold = np.mean(values)

for i, sublist in enumerate(keyword_single):
    lista_k_w_n = list(set([word for sentence in filtered_k_w_n[i] for word in sentence.split()]))
    for tupla in sublist:
        if tupla[1] >= threshold and tupla[0] not in lista_k_w_n:
            filtered_k_w_n[i].append(tupla[0])

keywords = filtered_k_w_n      

In [ ]:
keywords[1]

In [22]:
# Grabar
with open(PATH+f'modelos/keywords_{chunk}.json', 'w') as file:
    json.dump(keywords, file)

#### BOW - Armado del vocabulario con las entidades y keywords

In [23]:
# Unificar Entities + Keywords + Keywords with neighboards
vocab = list(set().union(*entities, *keywords))
len(vocab)

6826

In [24]:
# Guardar vocabulario
with open(PATH+f'modelos/vocabulary_{chunk}.json', 'w') as file:
    json.dump(vocab, file)

### Preprocesar las noticias

In [7]:
clean_data = Cleaning_text()

proc_data = []
for data_in in tqdm(data):
    aux = clean_data.unicode(data_in)
    aux = clean_data.urls(aux)
    aux = clean_data.simbols(aux)
    aux = clean_data.escape_sequence(aux)
    aux = " ".join([ word for word in aux.split() if word.lower() not in SPANISH_STOPWORDS_SPECIAL])
    proc_data.append(aux)


100%|██████████| 1000/1000 [00:00<00:00, 1175.35it/s]


In [8]:
# Grabar
with open(PATH+f'modelos/proc_data_{chunk}.json', 'w') as file:
    json.dump(proc_data, file)

In [22]:
proc_data[60]

'punto mira preocupantes informaciones salud tiempos Bertín Osborne reaparecido Semana Santa confirmando recuperación marcha camino comenzar tratamiendo inyección vitaminas superar secuelas arrastra contagió Covid principios enero entorno reveló enfermedad dejado defensas sistema inmune debilitado reposo seguido finca Sevilla semanas frutos días dejaba comiendo cochinillo familiares popular venta localidad gaditana Medina Sidonia Semana Santa residencia sevillana puntos Andalucía preguntado hija Claudia Osborne llegada aeropuerto Madrid disfrutar vacaciones marido José Entrecanales pequeñas Micaela Violeta nació 22 enero lejos país rostro cansado llevando bebé pegado pecho mochila porteadora hijo presidente Acciona llevaba primogénita brazos coach mostrado discreta cámaras revelado sonrisa padre gracias'

### Guardar noticias en el indice news de la base

In [25]:
# configurar  batch_size = ( ej.: 5000 ) si se supera el limite 100MB en elasticsearch por operacion
index_name = 'news'
bulk_data = []

for idx, text_news in tqdm(enumerate(data)):
    doc = {
        'index': {
            '_index': index_name,
            '_id': int(df_parquet.index[idx])
        }
    }
    reg = {
        'title': str(df_parquet.iloc[idx].in__title),
        'news' : str(text_news), 
        'author': str(df_parquet.iloc[idx]['Author Name']),
        'vector': None,
        'keywords' : keywords[idx],
        'entities' : entities[idx],
        'created_at': parse(str(df_parquet.iloc[idx]['start_time_local'])).isoformat(),
        'process': False
    }
    bulk_data.append(json.dumps(doc))
    bulk_data.append(json.dumps(reg))

# Convertir la lista en un solo string separado por saltos de línea
bulk_request_body = '\n'.join(bulk_data) + '\n'

# Enviar la solicitud bulk
response = os_client.bulk(body=bulk_request_body)

if response['errors']:
    print("Errores encontrados al insertar los documentos")
else:
    print("Documentos insertados correctamente")


1000it [00:00, 2004.04it/s]


Documentos insertados correctamente


#### Funciones de pruebas

In [26]:
# Encontrar la posicion en el df segun su ID
df_parquet.index.get_loc(105640350)

211

In [27]:
def funcion_aux(ID):
    keywords_df = df_parquet[df_parquet.index==ID]['Keyword Name'].values[0]
    entities_df = df_parquet[df_parquet.index==ID]['Entity Name'].values[0]
    fila = df_parquet.index.get_loc(ID)
    print(f"Noticia ID: {ID} {df_parquet[df_parquet.index==ID]['in__title'].values}\n")
    print(f"Entities de dataframe: {entities_df}")
    print(f"Keywords de dataframe: {keywords_df}")
    print("-"*80)
    print(f"Fila: {fila}")
    print(f"Entities calculadas: {entities[fila]}")
    print(f"Keywords calculadas: {filtered_k_w_n[fila]}")

funcion_aux(105638862)

Noticia ID: 105638862 ['Bitcoin: cómo se ha movido en el mercado este 2 de abril']

Entities de dataframe: ['Elon Musk' 'Bloomberg']
Keywords de dataframe: ['flujo' 'precio' 'moneda legal' 'mercado' 'bitcoin']
--------------------------------------------------------------------------------
Fila: 15
Entities calculadas: ['El Salvador', 'San Salvador']
Keywords calculadas: ['bitcoin', 'criptomonedas', 'mercado', 'criptomoneda', 'valor']
